In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import missingno as msno
import math
from matplotlib.gridspec import GridSpec


# Preprocessing
from sklearn.preprocessing import MinMaxScaler
# Algorithms
from minisom import MiniSom # This is for the unsupervised clustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


In [2]:
df = pd.read_csv("data\\2013_01_01_2014_01_01_day_temp_filtered.csv")

In [3]:
df

,DateTime,LCLid,stdorToU,KWH/hh,Acorn,Acorn_grouped,Day,air_temperature,Month
0,2013-01-01 00:00:00,MAC000295,Std,0.157,ACORN-E,Affluent,Tuesday,48.02,January
1,2013-01-01 00:30:00,MAC000295,Std,0.135,ACORN-E,Affluent,Tuesday,47.84,January
2,2013-01-01 01:00:00,MAC000295,Std,0.159,ACORN-E,Affluent,Tuesday,47.66,January
3,2013-01-01 01:30:00,MAC000295,Std,0.135,ACORN-E,Affluent,Tuesday,47.39,January
4,2013-01-01 02:00:00,MAC000295,Std,0.157,ACORN-E,Affluent,Tuesday,47.12,January
...,...,...,...,...,...,...,...,...,...
3164395,2014-01-01 21:30:00,MAC004193,Std,0.807,ACORN-D,Affluent,Wednesday,42.80,January
3164396,2014-01-01 22:00:00,MAC004193,Std,0.751,ACORN-D,Affluent,Wednesday,42.80,January
3164397,2014-01-01 22:30:00,MAC004193,Std,0.476,ACORN-D,Affluent,Wednesday,42.80,January
3164398,2014-01-01 23:00:00,MAC004193,Std,0.447,ACORN-D,Affluent,Wednesday,42.80,January


In [63]:
df_for_np = df.copy(deep = True) # Make a copy of the df to transform.

In [64]:
df_for_np = df_for_np.drop(['DateTime', 'LCLid', 'Acorn'], axis=1) # Drop non-numerical rows.
df_for_np = pd.get_dummies(df_for_np) # Make dummies.
scaler = MinMaxScaler()
df_for_np = scaler.fit_transform(df_for_np) # scale data.

In [66]:
df_for_np.shape

(3164400, 26)

In [67]:
n_meters = len(df.LCLid.unique())
n_meters
n_cols = 26
n_cols
n_rows = 17580

In [68]:
arr = df_for_np.reshape(n_meters, n_rows, n_cols)

In [69]:
arr

array([[[0.02541687, 0.34564644, 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.02185527, 0.34300792, 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.02574065, 0.34036939, 1.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.07236523, 0.26912929, 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.06006152, 0.26912929, 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.01376073, 0.30738786, 1.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.03577789, 0.34564644, 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.03092116, 0.34300792, 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.02023636, 0.34036939, 1.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.05617614, 0.26912929, 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.0

In [71]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.datasets import CachedDatasets

C:\Users\jesse\anaconda3\lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
C:\Users\jesse\anaconda3\lib\site-packages\tslearn\clustering\kmeans.py:16: UserWarning: Scikit-learn <0.24 will be deprecated in a future release of tslearn
  warnings.warn(


In [72]:
# Perform clustering for each metric
y_preds = []

km = TimeSeriesKMeans(n_clusters=6, random_state=0, metric="dtw")
y_preds.append(km.fit_predict(arr))


MemoryError: Unable to allocate 2.30 GiB for an array with shape (17580, 17580) and data type float64

In [ ]:
for yi in range(3):
    plt.subplot(3, 3, 4 + yi)
    for xx in arr[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(km.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-4, 4)
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 1:
        plt.title("DBA $k$-means")